In [ ]:
#Upload the file
from google.colab import files
files.upload()

Saving Online Retail.xlsx to Online Retail.xlsx
Buffered data was truncated after reaching the output size limit.

In [ ]:
#Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pandas as pd
!pip install ydata-profiling
from pandas_profiling import ProfileReport

In [ ]:
#Read the file
Data= pd.read_excel("Online Retail.xlsx")
print(Data.shape)

In [ ]:
Data.head()

In [ ]:
# Fill missing values with backward fill, then forward fill
Data['Description'] = Data['Description'].bfill().ffill()
Data['CustomerID']  = Data['CustomerID'].bfill().ffill()
Data['Country']     = Data['Country'].bfill().ffill()
Data['Quantity']    = Data['Quantity'].bfill().ffill()
Data['UnitPrice']   = Data['UnitPrice'].bfill().ffill()

# Ensure correct data types
Data['InvoiceNo']   = Data['InvoiceNo'].astype(str)
Data['StockCode']   = Data['StockCode'].astype(str)
Data['Description'] = Data['Description'].astype(str)
Data['Quantity']    = Data['Quantity'].astype(int)
Data['InvoiceDate'] = pd.to_datetime(Data['InvoiceDate'])
Data['UnitPrice']   = Data['UnitPrice'].astype(float)
Data['CustomerID']  = Data['CustomerID'].astype(int)
Data['Country']     = Data['Country'].astype(str)

# Final check
print(" Preprocessing completed")
print("Shape:", Data.shape)
print("Missing values:\n", Data.isnull().sum())
print("\nData types:")
print(Data.dtypes)

In [ ]:
report = ProfileReport(Data)
report

In [ ]:


# Reference date = 1 day after last purchase
ref_date = Data["InvoiceDate"].max() + dt.timedelta(days=1)

# Recency
recency = Data.groupby("CustomerID")["InvoiceDate"].max().reset_index()
recency["Recency"] = (ref_date - recency["InvoiceDate"]).dt.days
recency.drop(columns=["InvoiceDate"], inplace=True)

# Frequency
frequency = Data.groupby("CustomerID")["InvoiceNo"].nunique().reset_index()
frequency.rename(columns={"InvoiceNo": "Frequency"}, inplace=True)

# Monetary
Data['TotalPrice'] = Data['Quantity'] * Data['UnitPrice']
monetary = Data.groupby("CustomerID")["TotalPrice"].sum().reset_index()
monetary.rename(columns={"TotalPrice": "Monetary"}, inplace=True)

# Merge all into one RFM table
rfm = recency.merge(frequency, on="CustomerID").merge(monetary, on="CustomerID")

# Show result
print("RFM Table Created")
rfm.head()

In [ ]:
# RFM scoring (short & safe)
rfm['R'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1]).astype(int)
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1,2,3,4,5]).astype(int)
rfm['M'] = pd.qcut(rfm['Monetary'].rank(method="first"), 5, labels=[1,2,3,4,5]).astype(int)

# Combine into score + segment
rfm['RFM_Score'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['RFM_Sum'] = rfm[['R','F','M']].sum(axis=1)

rfm['Segment'] = np.select(
    [rfm['RFM_Sum'] >= 15, rfm['RFM_Sum'] >= 10, rfm['RFM_Sum'] >= 5],
    ['Champions', 'Loyal', 'Need Attention'],
    'At Risk'
)

# Show results
print(rfm[['CustomerID','Recency','Frequency','Monetary','Segment']].head())
print("\nSegment counts:\n", rfm['Segment'].value_counts())


In [ ]:
print(Data.shape
      )

In [ ]:
# Bar chart of segment counts
plt.figure(figsize=(6,4))
rfm['Segment'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title("Customer Segments (RFM)")
plt.xlabel("Segment")
plt.ylabel("Number of Customers")
plt.xticks(rotation=45)
plt.show()

# Heatmap of R vs F (optional)
rfm_pivot = rfm.pivot_table(index='R', columns='F', values='CustomerID', aggfunc='count').fillna(0)

plt.figure(figsize=(6,4))
sns.heatmap(rfm_pivot, cmap="Blues", annot=True, fmt=".0f")
plt.title("Heatmap of Customers (R vs F)")
plt.ylabel("Recency Score")
plt.xlabel("Frequency Score")
plt.show()


In [ ]:
def suggest_action(segment):
    if segment == 'Champions':
        return 'VIP rewards, early access, exclusive offers'
    elif segment == 'Loyal':
        return 'Loyalty programs, personalized recommendations'
    elif segment == 'Need Attention':
        return 'Engagement emails, small discounts, reminders'
    elif segment == 'At Risk':
        return 'Win-back campaigns, larger discounts, retargeting'
    else:
        return 'No action'

segments = rfm['Segment'].unique()

for seg in segments:
    print(f"{seg}: {suggest_action(seg)}")


In [ ]:
# Loop over each customer and print suggestion
for cust_id, seg in zip(rfm['CustomerID'], rfm['Segment']):
    print(f"Customer {cust_id} ({seg}): {suggest_action(seg)}")
